In [1]:
pip install pandas psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.3 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [198]:
import pandas as pd
import psycopg2 as pg

In [84]:
types_considered = ['Real estate agency',
                    'Real estate consultant',
                    'Real estate agent',
                    'Property investment',
                    'Property management company',
                    'Real estate agency',
                    'Real estate consultant',
                    'Real estate agent']

types_considered_str = ', '.join(f"'{t}'" for t in types_considered)
type_filter = f"type in ({types_considered_str})"


### Top real state companies by city

The function "get_company_profiles_city_state_type_sorted_by_score" allows a user to retrive top n real state companies, by city or state, ordered by company_score

In [77]:
def get_company_profiles_city_state_type_sorted_by_score(
    engine, 
    city=None, 
    state=None, 
    type=None, 
    limit=None
):
    """
    Retrieves company profiles from the database with optional filtering by city, state, and type.
    """
    
    query = """
    SELECT 
        dc.name, 
        dc.type, 
        dc.city, 
        dc.state, 
        dc.rating, 
        dc.reviews, 
        fc.company_profile_score AS company_score
    FROM 
        curated_dim_company_profiles_google_maps dc
    INNER JOIN
        curated_score_company_profiles_google_maps fc
    ON
        dc.google_id = fc.google_id
    WHERE 
        verified = 1
    """
    params = {}
    
    if type is not None:
        type = [val.upper() for val in type] if isinstance(type, list) else type.upper()
        type_clause, type_params = add_filter("UPPER(dc.type)", type, "type")
        query += type_clause
        params.update(type_params)
        
    if city is not None:
        city = [val.capitalize() for val in city] if isinstance(city, list) else city.capitalize()
        city_clause, city_params = add_filter("dc.city", city, "city")
        query += city_clause
        params.update(city_params)
        
    if state is not None:
        state = [val.capitalize() for val in state] if isinstance(state, list) else state.capitalize()
        state_clause, state_params = add_filter("dc.state", state, "state")
        query += state_clause
        params.update(state_params)
    
    query += " ORDER BY fc.company_profile_score DESC"
    
    if limit is not None:
        query += " LIMIT %(limit)s"
        params['limit'] = limit

    df = pd.read_sql(query, engine, params=params)
    
    return df

def add_filter(field, value, param_name):
    if isinstance(value, list):
        placeholders = ', '.join([f"%({param_name}_{i})s" for i in range(len(value))])
        return f" AND {field} IN ({placeholders})", {f"{param_name}_{i}": v for i, v in enumerate(value)}
    else:
        return f" AND {field} = %({param_name})s", {param_name: value}

##### Miami

In [79]:
get_company_profiles_city_state_type_sorted_by_score(engine, city="Miami", type=types_considered, limit=5)

/tmp/ipykernel_11092/4129417398.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine, params=params)


,name,type,city,state,rating,reviews,company_score
0,Homeinc- Miami,Real estate agent,Miami,Florida,5.0,222.0,0.871008
1,"Melissa Adams, RE/MAX Advance Realty",Real estate agent,Miami,Florida,5.0,123.0,0.855385
2,Miami Condo Investments,Real estate agency,Miami,Florida,4.9,275.0,0.848471
3,Yeimy Ortiz PA - Real Estate Broker Associate,Real estate agent,Miami,Florida,5.0,167.0,0.848024
4,Miami Realty Solution Group,Real estate consultant,Miami,Florida,4.9,152.0,0.843714


##### Dallas

In [81]:
get_company_profiles_city_state_type_sorted_by_score(engine, city="Dallas", type=types_considered, limit=5)

/tmp/ipykernel_11092/4129417398.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine, params=params)


,name,type,city,state,rating,reviews,company_score
0,Dan Harker - EmpowerHome Team Dallas,Real estate agency,Dallas,Texas,5.0,509.0,0.841380
1,Dwellinc Apartment Locating & Realty,Real estate agency,Dallas,Texas,5.0,491.0,0.839989
2,Livian DFW,Real estate agency,Dallas,Texas,5.0,256.0,0.814846
3,Terratino,Real estate agent,Dallas,Texas,4.9,300.0,0.805964
4,Templeton Real Estate Group,Real estate agency,Dallas,Texas,5.0,191.0,0.803556


##### Seatle

In [82]:
get_company_profiles_city_state_type_sorted_by_score(engine, city="Seattle", type=types_considered, limit=5)

/tmp/ipykernel_11092/4129417398.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine, params=params)


,name,type,city,state,rating,reviews,company_score
0,Every Door Real Estate,Real estate agency,Seattle,Washington,5.0,791.0,0.858421
1,Geoff Hill,Real estate agent,Seattle,Washington,5.0,133.0,0.789632
2,Danny Greco Homes,Real estate agent,Seattle,Washington,5.0,122.0,0.786315
3,TRI STAR Team | RE/MAX NW,Real estate agency,Seattle,Washington,5.0,105.0,0.780556
4,SoundTeamRealty,Real estate agent,Seattle,Washington,5.0,101.0,0.779067


### Diversity and acessibility by city

##### Accessibility

In [98]:
query = f"""
SELECT 
    dc.city,
    (SUM(dc.wheelchair_accessible_entrance) / COUNT(*)) * 100 AS "%Companies with Wheelchair Accessibility"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
GROUP BY dc.city;
"""

In [99]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,city,%Companies with Wheelchair Accessibility
0,Dallas,76.798561
1,Seattle,67.450059
2,Miami,71.907101


Dallas: 76.80% of companies are wheelchair accessible, making it the city with the highest accessibility rate in the sample.
Miami: Close behind Dallas, 71.91% of companies in Miami have wheelchair-accessible entrances.
Seattle: 67.45% of companies in Seattle are accessible, showing slightly lower but still significant accessibility.
This analysis indicates that over two-thirds of the companies in all three cities provide wheelchair access, reflecting a positive trend towards inclusivity. 

##### Trangender

In [128]:
query = f"""
SELECT 
    dc.city,
    (SUM(dc.transgender_safespace) / COUNT(*)) * 100 AS "%Companies with transgender safespace"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
GROUP BY dc.city;
"""

In [129]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,city,%Companies with transgender safespace
0,Dallas,1.798561
1,Seattle,5.287897
2,Miami,1.875837


In this analysis of companies providing transgender safespaces, the data reveals that:

Seattle has the highest percentage of companies offering transgender safespaces, at 5.29%.
Miami comes next with 1.88%.
Dallas has the lowest percentage, at 1.80%.
It’s important to note that these percentages do not necessarily indicate that companies in Dallas or Miami are less inclusive or transphobic. The low percentages could simply reflect the fact that many companies may not have filled out or updated this specific field in their profiles.

### Share of each company type

In [133]:
query = f"""
SELECT 
    type,
    COUNT(*) AS total_companies,
    ROUND((COUNT(*)::decimal / SUM(COUNT(*)) OVER ()) * 100, 2) AS "%Representation"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
GROUP BY 
    type
ORDER BY 
    "%Representation" DESC;
"""

In [134]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,type,total_companies,%Representation
0,Real estate agency,2006,47.74
1,Real estate agent,1807,43.00
2,Real estate consultant,312,7.43
3,Property management company,76,1.81
4,Property investment,1,0.02


This breakdown highlights that real estate agencies and agents dominate the industry, while other types of real estate companies, such as consultants, management companies, and investors, hold much smaller shares.

### Most common company working hours

##### Weekedays

In [143]:
query = f"""
SELECT 
    weekdays_working_hours,
    COUNT(*) AS total_companies,
    ROUND((COUNT(*)::decimal / SUM(COUNT(*)) OVER ()) * 100, 2) AS "%Representation"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
    AND weekdays_working_hours is not null
GROUP BY 
    weekdays_working_hours
ORDER BY 
    "%Representation" DESC
LIMIT 10
"""

In [144]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,weekdays_working_hours,total_companies,%Representation
0,Open 24 hours,726,21.68
1,9AM-5PM,708,21.15
2,9AM-6PM,295,8.81
3,8AM-8PM,204,6.09
4,9AM-7PM,180,5.38
5,9AM-8PM,111,3.32
6,9AM-9PM,96,2.87
7,8AM-6PM,91,2.72
8,8AM-7PM,86,2.57
9,8AM-5PM,82,2.45


The two most common working hours setups are either 24/7 operations or the traditional 9 AM - 5 PM schedule, showing a balance between companies that are always available and those that follow conventional working hours. The rest of the companies show variability with extended business hours, suggesting some flexibility to accommodate customers with different schedules.

##### Saturdays

In [149]:
query = f"""
SELECT 
    saturday_working_hours,
    COUNT(*) AS total_companies,
    ROUND((COUNT(*)::decimal / SUM(COUNT(*)) OVER ()) * 100, 2) AS "%Representation"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
    AND saturday_working_hours is not null
GROUP BY 
    saturday_working_hours
ORDER BY 
    "%Representation" DESC
LIMIT 10
"""

In [150]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,saturday_working_hours,total_companies,%Representation
0,Closed,995,29.72
1,Open 24 hours,719,21.48
2,9AM-5PM,187,5.59
3,8AM-8PM,178,5.32
4,9AM-7PM,115,3.43
5,9AM-6PM,114,3.41
6,9AM-9PM,89,2.66
7,9AM-8PM,68,2.03
8,8AM-9PM,65,1.94
9,10AM-4PM,54,1.61


##### Sundays

In [151]:
query = f"""
SELECT 
    sunday_working_hours,
    COUNT(*) AS total_companies,
    ROUND((COUNT(*)::decimal / SUM(COUNT(*)) OVER ()) * 100, 2) AS "%Representation"
FROM 
    curated_dim_company_profiles_google_maps dc
WHERE 1=1
    AND {type_filter}
    AND sunday_working_hours is not null
GROUP BY 
    sunday_working_hours
ORDER BY 
    "%Representation" DESC
LIMIT 10
"""

In [152]:
pd.read_sql(query, engine)

/tmp/ipykernel_11092/3752518698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, engine)


,sunday_working_hours,total_companies,%Representation
0,Closed,1447,43.22
1,Open 24 hours,695,20.76
2,8AM-8PM,151,4.51
3,9AM-5PM,124,3.70
4,9AM-9PM,72,2.15
5,9AM-6PM,70,2.09
6,9AM-7PM,67,2.00
7,8AM-9PM,55,1.64
8,9AM-8PM,55,1.64
9,8AM-7PM,43,1.28


This trend highlights the general reduction in business operations over the weekend, particularly on Sundays. However, there is still a notable proportion of companies that provide services throughout the weekend, with many offering 24-hour service.

### Most used adjectives for the top 50 best real state companies with reviews (sentimental analysis)

In [166]:
query = f"""
WITH adjectives as (
    SELECT google_id, adjectives
    FROM
        curated_fact_customer_reviews_google
    GROUP BY
        google_id
)

with top_100_companies (
SELECT 
    dc.*, sc.company_profile_score as score
FROM 
    curated_dim_company_profiles_google_maps dc
INNER JOIN
    curated_score_company_profiles_google_maps sc
ON
    sc.google_id = fc.google_id
INNER JOIN
    curated_fact_customer_reviews_google fcr
ON
    fcr.google_id = fc.google_id
    
LEFT JOIN
    reviews_scorea
WHERE 1=1
    AND {type_filter}
ORDER BY company_profile_score DESC)

SELECT distinct adjectives
FROM top_100_companies left


"""

In [194]:
query = f"""
WITH reviews_score as (
    SELECT google_id, AVG(user_review_score) as review_score
    FROM
        curated_score_customer_reviews_google
    GROUP BY
        google_id
),

top_50_companies_with_reviews as (
SELECT 
    dc.google_id, fc.company_profile_score as company_score
FROM 
    curated_dim_company_profiles_google_maps dc
INNER JOIN
    curated_score_company_profiles_google_maps fc
ON
    dc.google_id = fc.google_id
INNER JOIN
    reviews_score rs
ON rs.google_id = fc.google_id
WHERE 1=1
    AND review_score is not null
    AND {type_filter}

ORDER BY 
    company_profile_score DESC
LIMIT 50)

SELECT distinct adjectives
FROM 
    top_50_companies_with_reviews tc
LEFT JOIN
    curated_fact_customer_reviews_google fc
ON
    tc.google_id = fc.google_id
"""

In [195]:
df_adjectives = pd.read_sql(query, engine)

/tmp/ipykernel_11092/1851159711.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_adjectives = pd.read_sql(query, engine)


In [196]:
def count_adjectives(df, col):
    """
    """
    
    adj_series = (
        df[col]
        .dropna()
        .astype(str)
        .str.split(',')
        .explode()
        .str.strip()
    )
    
    count = adj_series.value_counts().reset_index()
    count.columns = ['adjective', 'count']
    
    return count


In [197]:
count_adjectives(df_adjectives, "adjectives").head(10)

,adjective,count
0,great,943
1,first,857
2,real,718
3,best,591
4,professional,575
5,knowledgeable,463
6,new,453
7,able,386
8,responsive,358
9,easy,341


This analysis indicates that customers generally have a very positive perception of the top real estate companies. Words like "great," "best," and "professional" reflect a high level of satisfaction. Additionally, terms like "knowledgeable" and "responsive" suggest that customers value expertise and timely responses from these companies.